# Plantilla profesional: Portafolio conectado a Refinitiv (con fallback a yfinance)

Este notebook es una plantilla completa para:
- Conectar a Refinitiv (sección con placeholders y notas de configuración).
- Descargar históricos de precios para una lista configurable de activos (usa Refinitiv si está disponible o yfinance como fallback para pruebas).
- Generar señales (ej. SMA rápida vs lenta), backtesting diario, métricas por activo, y construcción de portafolio equiponderado y optimizado (Markowitz).
- Visualizaciones y exportación de resultados a CSV.

Notas: Debes disponer de credenciales y permisos para usar Refinitiv desde tu entorno; en ausencia de credenciales el notebook usará `yfinance` para demostración/local testing.

## 1) Instalación de dependencias

El siguiente bloque intenta importar las librerías necesarias y, si faltan, las instala automáticamente. En Jupyter esto puede llevar varios minutos la primera vez.

In [ ]:
import sys
import subprocess

def ensure(pkg):
    try:
        __import__(pkg)
    except Exception:
        print(f'Instalando {pkg}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

# Paquetes recomendados: refinitiv-data (si usas RDP), yfinance para fallback, plus scientific stack
for p in [
, 
, 
, 
, 
, 
]:
    # para import usamos nombres de módulo cuando difieren del nombre pip (ej: yfinance -> yfinance)
    ensure(p)

print('Dependencias comprobadas (puede requerir reiniciar el kernel si se instalaron paquetes).')

## 2) Configuración y autenticación Refinitiv

Aquí indicamos cómo configurar la conexión a Refinitiv. No ejecutes sin tus credenciales. Si no vas a usar Refinitiv ahora, el notebook caerá en `yfinance` como fuente alternativa.

- Documentación SDK RDP (Refinitiv Data Platform): consulta la guía oficial y la versión del SDK que uses (por ejemplo `refinitiv-data`).
- Métodos de autenticación: AppKey + token, OAuth, o Eikon desktop credentials — depende de tu licencia.

Modifica los placeholders en la celda de código para introducir tus credenciales de forma segura (mejor usar variables de entorno o un archivo `.env` local no versionado).


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Intentamos importar el cliente de Refinitiv; si no existe, activamos fallback a yfinance
USE_REFINITIV = False
refinitiv_client = None
try:
    # Import seguro: el nombre del paquete puede variar por versión; este es un ejemplo común
    from refinitiv.data import Content  # solo para probar import
    USE_REFINITIV = True
    # Aquí iría la inicialización real del cliente, p. ej.:
    # from refinitiv.data import Session; session = Session(app_key=..., ...)
    print('Módulo Refinitiv importado (recuerda inicializar sesión con tus credenciales).')
except Exception as e:
    print('Refinitiv SDK no disponible o no importó. Se usará yfinance como fallback. Detalle:', e)

# Para fallback: yfinance
import yfinance as yf

# Función helper para bajar precios con yfinance (safe)
def get_prices_yf(symbol, start=None, end=None, interval='1d'):
    df = yf.download(symbol, start=start, end=end, interval=interval, progress=False)
    if df.empty:
        raise ValueError(f'No hay datos para {symbol} (yfinance)')
    # Normalizar nombre de columna a 'close' usado en este notebook
    df = df.rename(columns={col: col.lower() for col in df.columns})
    if 'close' not in df.columns:
        # intenta con 'Adj Close' si falta
        if 'adj close' in df.columns:
            df['close'] = df['adj close']
    df = df[['close']].dropna()
    return df

def get_prices(symbol, start=None, end=None, source_priority=('refinitiv','yfinance')):
    
,
,
,

    for src in source_priority:
        if src == 'refinitiv' and USE_REFINITIV:
            try:
                # === Placeholder: implementación específica según tu SDK/versión ===
                # Ejemplo conceptual (no ejecutable sin adaptar):
                # resp = refinitiv_client.get_historical_prices(instrument=symbol, start=start, end=end)
                # df = resp.to_dataframe()
                # df = df.rename(columns={'Close': 'close'})[['close']]
                raise NotImplementedError('Implementa aquí la llamada específica a Refinitiv con tus credenciales y formato.')
            except Exception as e:
                print(f'Error Refinitiv para {symbol}:', e)
                continue
        if src == 'yfinance':
            try:
                return get_prices_yf(symbol, start=start, end=end)
            except Exception as e:
                print(f'Error yfinance para {symbol}:', e)
                continue
    raise RuntimeError(f'No fue posible descargar precios para {symbol} con las fuentes solicitadas.')

## 3) Lista de activos (personalizable)

Hemos tomado las empresas que indicaste. Dependiendo del proveedor (Refinitiv vs yfinance) el símbolo puede variar; ajusta `symbols_map` si lo necesitas.


In [ ]:
# Lista de activos solicitada por el usuario (etiqueta amigable -> default símbolo para yfinance)
symbols_map = {
    'JPM': 'JPM',
    'RY': 'RY.TO',                # Royal Bank of Canada (TSX) - ajustar si usas otra fuente
    'MITSUB': '8058.T',          # Mitsubishi Corp (TSE) - usa el código apropiado según proveedor
    'BA': 'BA',
    'LMT': 'LMT',
    'NVDA': 'NVDA',
    'MSFT': 'MSFT',
    'SAND': 'SAND.TO',           # Sandstorm Gold (TSX) - ajustar si necesitas otra bolsa
    'VLO': 'VLO'
}

# Puedes editar esta lista. El loop de descarga usará estos símbolos.
symbols_map

## 4) Señales: Estrategia SMA (rápida vs lenta)

Definimos la estrategia SMA clásica y funciones para métricas y evaluación por activo.

In [ ]:
FAST = 21
SLOW = 63

def build_sma_strategy(df, fast=FAST, slow=SLOW):
    data = df.copy()
    data[f'SMA_{fast}'] = data['close'].rolling(fast).mean()
    data[f'SMA_{slow}'] = data['close'].rolling(slow).mean()
    data['position'] = np.where(data[f'SMA_{fast}'] > data[f'SMA_{slow}'], 1, -1)
    data['ret'] = np.log(data['close'] / data['close'].shift(1))
    data['strat_ret'] = data['position'].shift(1) * data['ret']
    data = data.dropna()
    return data

def max_drawdown(cum_returns):
    roll_max = cum_returns.cummax()
    dd = (cum_returns / roll_max) - 1.0
    return dd.min()

def annualized_vol(daily_returns):
    return daily_returns.std() * np.sqrt(252)

def cagr(total_return, n_days):
    years = n_days / 252
    if years <= 0:
        return np.nan
    return (1 + total_return) ** (1 / years) - 1

def sharpe_ratio(daily_returns, rf=0.0):
    vol = annualized_vol(daily_returns)
    if vol == 0 or np.isnan(vol):
        return np.nan
    return (daily_returns.mean() * 252 - rf) / vol

def count_trades(position_series):
    pos = position_series.dropna()
    changes = (pos != pos.shift(1))
    return int(changes.sum())

from dataclasses import dataclass
@dataclass
class StrategyMetrics:
    ticker: str
    start: pd.Timestamp
    end: pd.Timestamp
    n_days: int
    total_return_bh: float
    total_return_strat: float
    cagr_bh: float
    cagr_strat: float
    vol_bh: float
    vol_strat: float
    sharpe_bh: float
    sharpe_strat: float
    max_dd_bh: float
    max_dd_strat: float
    calmar_bh: float
    calmar_strat: float
    trades: int
    win_rate: float
    pct_long: float
    pct_short: float

def build_metrics(ticker, data):
    cum_bh = data['ret'].cumsum().apply(np.exp)
    cum_strat = data['strat_ret'].cumsum().apply(np.exp)
    total_return_bh = cum_bh.iloc[-1] - 1
    total_return_strat = cum_strat.iloc[-1] - 1
    n_days = data.shape[0]
    cagr_b = cagr(total_return_bh, n_days)
    cagr_s = cagr(total_return_strat, n_days)
    vol_b = annualized_vol(data['ret'])
    vol_s = annualized_vol(data['strat_ret'])
    sharpe_b = sharpe_ratio(data['ret'])
    sharpe_s = sharpe_ratio(data['strat_ret'])
    max_dd_b = max_drawdown(cum_bh)
    max_dd_s = max_drawdown(cum_strat)
    calmar_b = np.nan if max_dd_b == 0 else cagr_b / abs(max_dd_b)
    calmar_s = np.nan if max_dd_s == 0 else cagr_s / abs(max_dd_s)
    trades = count_trades(data['position'])
    win_rate = (data['strat_ret'] > 0).mean()
    pct_long = (data['position'] == 1).mean()
    pct_short = (data['position'] == -1).mean()
    return StrategyMetrics(ticker=ticker, start=data.index[0], end=data.index[-1], n_days=n_days,
                           total_return_bh=total_return_bh, total_return_strat=total_return_strat,
                           cagr_bh=cagr_b, cagr_strat=cagr_s, vol_bh=vol_b, vol_strat=vol_s,
                           sharpe_bh=sharpe_b, sharpe_strat=sharpe_s, max_dd_bh=max_dd_b,
                           max_dd_strat=max_dd_s, calmar_bh=calmar_b, calmar_strat=calmar_s,
                           trades=trades, win_rate=win_rate, pct_long=pct_long, pct_short=pct_short)

def metrics_to_frame(metrics_list):
    rows = []
    for m in metrics_list:
        rows.append({
            'Ticker': m.ticker,
            'Start': m.start.date(),
            'End': m.end.date(),
            'Días': m.n_days,
            'TotRet_BH': m.total_return_bh,
            'TotRet_Strat': m.total_return_strat,
            'CAGR_BH': m.cagr_bh,
            'CAGR_Strat': m.cagr_strat,
            'Vol_BH': m.vol_bh,
            'Vol_Strat': m.vol_strat,
            'Sharpe_BH': m.sharpe_bh,
            'Sharpe_Strat': m.sharpe_strat,
            'MaxDD_BH': m.max_dd_bh,
            'MaxDD_Strat': m.max_dd_strat,
            'Calmar_BH': m.calmar_bh,
            'Calmar_Strat': m.calmar_strat,
            'Trades': m.trades,
            'WinRate': m.win_rate,
            'Pct_Long': m.pct_long,
            'Pct_Short': m.pct_short
        })
    return pd.DataFrame(rows)


## 5) Descarga de datos y cálculo por activo

Este bloque descarga precios para cada activo, construye la estrategia SMA y guarda los resultados en `all_data` y `metrics_list`.

In [ ]:
all_data = {}
metrics_list = []

for label, sym in symbols_map.items():
    print(f'Procesando {label} ({sym})...')
    try:
        dfp = get_prices(sym)
        sdf = build_sma_strategy(dfp, fast=FAST, slow=SLOW)
        m = build_metrics(label, sdf)
        all_data[label] = sdf
        metrics_list.append(m)
    except Exception as e:
        print(f'Error con {label}:', e)

metrics_df = metrics_to_frame(metrics_list)
metrics_df = metrics_df.sort_values('CAGR_Strat', ascending=False).reset_index(drop=True)
metrics_df

## 6) Visualización individual y comparativa

Puedes cambiar `VIEW` para inspeccionar cualquier activo cargado.

In [ ]:
VIEW = 'NVDA'  # cambia según quieras ver otro ticker de symbols_map keys
if VIEW in all_data:
    dv = all_data[VIEW]
    fig, axes = plt.subplots(2,1, figsize=(12,8), sharex=True)
    axes[0].plot(dv.index, dv['close'], label='Close', alpha=0.6)
    axes[0].plot(dv.index, dv[f'SMA_{FAST}'], label=f'SMA {FAST}', alpha=0.8)
    axes[0].plot(dv.index, dv[f'SMA_{SLOW}'], label=f'SMA {SLOW}', alpha=0.8)
    axes[0].set_title(f'{VIEW} - Precio y SMAs')
    axes[0].legend(loc='upper left')
    cum_bh = dv['ret'].cumsum().apply(np.exp)
    cum_str = dv['strat_ret'].cumsum().apply(np.exp)
    axes[1].plot(cum_bh.index, cum_bh, label='Buy & Hold', color='gray')
    axes[1].plot(cum_str.index, cum_str, label='Estrategia', color='purple')
    axes[1].set_title('Curvas de capital (base 1.0)')
    axes[1].legend(loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print('Ticker no cargado en all_data. Ejecuta la celda de descarga primero.')

## 7) Portafolio equiponderado y métricas agregadas

Construimos un portafolio simple con igual peso entre activos y comparamos Buy & Hold vs Estrategia.

In [ ]:
# Alineamos retornos por día
rets_bh = []
rets_str = []
for label, dfv in all_data.items():
    rets_bh.append(dfv['ret'].rename(label))
    rets_str.append(dfv['strat_ret'].rename(label))

rets_bh = pd.concat(rets_bh, axis=1).dropna(how='all')
rets_str = pd.concat(rets_str, axis=1).reindex(index=rets_bh.index).fillna(0)

port_ret_bh = rets_bh.mean(axis=1)
port_ret_str = rets_str.mean(axis=1)

cum_bh_port = port_ret_bh.cumsum().apply(np.exp)
cum_str_port = port_ret_str.cumsum().apply(np.exp)

plt.figure(figsize=(12,5))
plt.plot(cum_bh_port, label='Port Buy & Hold', color='black')
plt.plot(cum_str_port, label='Port Estrategia', color='teal')
plt.title('Portafolio Equiponderado (base 1.0)')
plt.legend()
plt.show()

port_metrics = {
    'TotRet_BH': cum_bh_port.iloc[-1]-1,
    'TotRet_Strat': cum_str_port.iloc[-1]-1,
    'CAGR_BH': cagr(cum_bh_port.iloc[-1]-1, len(cum_bh_port)),
    'CAGR_Strat': cagr(cum_str_port.iloc[-1]-1, len(cum_str_port)),
    'Vol_BH': annualized_vol(port_ret_bh),
    'Vol_Strat': annualized_vol(port_ret_str),
    'Sharpe_BH': sharpe_ratio(port_ret_bh),
    'Sharpe_Strat': sharpe_ratio(port_ret_str),
    'MaxDD_BH': max_drawdown(cum_bh_port),
    'MaxDD_Strat': max_drawdown(cum_str_port),
}
port_metrics

## 8) Optimización (Markowitz) - frontera eficiente

Usamos retornos diarios del portafolio (estrategia o buy & hold según preferencia) para calcular la frontera eficiente y obtener pesos Max Sharpe / Min Vol.

In [ ]:
from scipy.optimize import minimize

USE_STRAT = True  # cambia a False si quieres usar Buy&Hold en la optimización
returns_mkv = rets_str if USE_STRAT else rets_bh
mean_daily = returns_mkv.mean()
cov_daily = returns_mkv.cov()
mean_annual = mean_daily * 252
cov_annual = cov_daily * 252
rf = 0.0
n_assets = returns_mkv.shape[1]

def portfolio_perf(weights, mean_ret=mean_annual, cov=cov_annual, rf=rf):
    ret = np.dot(weights, mean_ret)
    vol = np.sqrt(weights @ cov @ weights)
    sharpe = (ret - rf) / vol if vol > 0 else np.nan
    return ret, vol, sharpe

def max_sharpe(short=False):
    bounds = ((-1,1) if short else (0,1),) * n_assets
    x0 = np.full(n_assets, 1/n_assets)
    cons = ({'type':'eq','fun':lambda w: np.sum(w)-1},)
    def neg_sharpe(w):
        return -portfolio_perf(w)[2]
    res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res

def min_vol(short=False):
    bounds = ((-1,1) if short else (0,1),) * n_assets
    x0 = np.full(n_assets, 1/n_assets)
    cons = ({'type':'eq','fun':lambda w: np.sum(w)-1},)
    def vol(w):
        return portfolio_perf(w)[1]
    res = minimize(vol, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res

res_ms = max_sharpe(short=False)
res_mv = min_vol(short=False)
ret_ms, vol_ms, sr_ms = portfolio_perf(res_ms.x)
ret_mv, vol_mv, sr_mv = portfolio_perf(res_mv.x)
print(f'Max Sharpe -> Ret: {ret_ms:.2%} Vol: {vol_ms:.2%} Sharpe: {sr_ms:.2f}')
print(f'Min Vol    -> Ret: {ret_mv:.2%} Vol: {vol_mv:.2%} Sharpe: {sr_mv:.2f}')

weights_df = pd.DataFrame({
    'Activo': returns_mkv.columns,
    'Peso_MaxSharpe': res_ms.x,
    'Peso_MinVol': res_mv.x
}).set_index('Activo')
weights_df

## 9) Exportar resultados y siguientes pasos

Guarda métricas y pesos. Recomendación: no versionar credenciales y guardarlas en variables de entorno o `.env` local.

In [ ]:
metrics_df.to_csv('metrics_individuales.csv', index=False)
weights_df.to_csv('weights_markowitz.csv')
print('Exportados: metrics_individuales.csv, weights_markowitz.csv')

---
### Notas finales
- Para activar Refinitiv reemplaza el placeholder en `get_prices` por las llamadas concretas de tu SDK e inicializa `refinitiv_client` con tus credenciales (mejor: usa variables de entorno).
- Si quieres, puedo integrar esa parte por ti: dime qué SDK usas exactamente (`refinitiv-data` versión X, `eikon`/`rdp` etc.) y puedo adaptar las llamadas con ejemplos ejecutables.
- También puedo añadir costos de transacción, slippage, y un módulo de métricas más avanzado (turnover, information ratio, sortingino, etc.) si lo deseas.
